In [ ]:
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import pdfo

In [ ]:
data = np.load("data/dataset/sel_voxels_from_4_ds.npy")
print(data.shape)

In [ ]:
cov_m = np.cov(data.T)

sn.heatmap(cov_m)
plt.show()

In [ ]:
print(np.min(cov_m))

In [ ]:
e_val, e_vec = np.linalg.eig(cov_m)

In [ ]:
plt.plot(e_val, marker='o')
plt.yscale("log")
plt.show()
plt.yscale("log")
plt.plot(e_val[:10], marker='o')
plt.show()

In [ ]:
cen = np.array([np.average(x) for x in data.T])

ti_seq = np.array([*list(range(50, 400, 25)),
          *list(range(400, 1000, 10)),
          1000, 1030, 1050, 1080, 1100, 1130, 1150, 1180, 1200, 1230, 1250, 1280, 1300, 1330, 1350, 1380,
          1400, 1450, 1500, 1550, 1600, 1650, 1700,
          1800, 1900, 2000, 2100, 2200, 2300, 2500, 3000])

plt.figure(figsize=(15, 10))
for i in range(6):
    plt.plot(ti_seq, e_vec[:,i], label="{}".format(i))
plt.legend()
plt.show()

Centroid

In [ ]:
plt.plot(ti_seq, cen)
plt.show()

In [ ]:
test_data = data[79]
decentr = test_data - cen

vec_count = 7
coefs = []
for i in range(105):
    l = np.linalg.norm(e_vec[:,i])
    coefs.append(sum(decentr * e_vec[:,i]) / (l ** 2))

rec = np.copy(cen)
for i in range(vec_count):
    rec += coefs[i] * e_vec[:,i]

s = np.copy(cen)
for i in range(6):
    s += coefs[i] * e_vec[:,i]

plt.plot(ti_seq, test_data, label="data")
plt.plot(ti_seq, rec, label="7")
plt.plot(ti_seq, s, label="6")
plt.legend()
plt.show()

In [ ]:
rec = np.copy(cen)
for i in range(6):
    rec += coefs[i] * e_vec[:,i]

plt.plot(ti_seq, test_data, label="data")
plt.plot(ti_seq, rec, label="trans")
plt.legend()
plt.show()

In [ ]:
plt.plot(coefs)
plt.title("Coefs")
plt.show()


In [ ]:
def model(TI,T1,M0=1.):
    return M0*np.abs(1.-2.*np.exp(-TI/T1))

def modcurve(T1,M0=1.):
    return model(ti_seq,T1,M0)

def curvecoord(curve,base):
    coord = np.dot(base.T,curve)
    bnorm = np.linalg.norm(base,axis=0)
    coord /= bnorm**2
    return coord

def curveproj(curve,base):
    coord = curvecoord(curve,base)
    proj = np.sum(base * coord,axis=1) 
    return coord,proj



In [ ]:
plt.plot(ti_seq,modcurve(1000,1200))
plt.plot(ti_seq,data[44])

In [ ]:
# mozna 7 ale 6 vypada lip
base = e_vec[:,:6]

In [ ]:

T1range = range(400,1800,5)
M1range = [100,200,300,500,1000]

plt.figure(figsize=(12,8))
for M1 in M1range:
    res = []
    for T1 in T1range:
        t1curve=modcurve(T1,M1)
        t1cen = t1curve - cen
        _,t1proj = curveproj(t1cen,base)
        t1res = t1proj - t1cen
        res.append(np.linalg.norm(t1res))
    
    plt.plot(T1range,res,label=str(M1))
    
plt.legend()
plt.show()
    

In [ ]:
# T1start = np.array([660,716,800,950,1180,1620])
T1start = np.array([650,710,810,1000,1250,1580])

In [ ]:
models = []
#for t1 in range(500,1900,200):
for t1 in T1start:
    models.append(modcurve(t1,1200))
    
models = np.array(models).T
avgm = np.average(models,axis=1)

plt.plot(ti_seq,avgm)
plt.show()

In [ ]:
M0 = 1200 * curvecoord(cen,avgm)
M0

In [ ]:
wcond = 0.1

def goodbase(T1s,M0,obase,cntr,opt=True):
    b = []
    cu = []
    co = []
    for t1 in T1s:
        t1curve=modcurve(t1,M0)
        t1cen = t1curve - cntr
        t1coord,t1proj = curveproj(t1cen,obase)
        b.append(t1proj)
        co.append(t1coord)
        cu.append(t1cen)
        
    t1coords = np.array(co).T
    t1curves = np.array(cu).T
    t1base = np.array(b).T
    
    t1res = np.linalg.norm(t1base - t1curves) / len(T1s)
#    resnorm = np.linalg.norm(t1base - t1curves,axis=0) * T1s / np.linalg.norm(T1s)
#    t1res = np.linalg.norm(resnorm) / len(T1s)
    t1cond = np.linalg.cond(t1base)
    
#    return t1res,t1cond
    if opt:
        return t1res + wcond * t1cond
    
    return t1res,t1cond,t1base,t1coords,t1res + wcond * t1cond

In [ ]:
res = []
for i in range(100):
    Tshift = np.random.standard_normal(size=len(T1start)) * 0.05 - 0.1
    T1_0 = T1start * (1.+Tshift)
    r = pdfo.newuoa(goodbase,T1_0,args=(M0,base,cen))
    if r.success:
        print(i,T1_0,'OK')
        res.append(r)
    else:
        print (i, T1_0, r.message)
    


In [ ]:
for r in sorted(res,key=lambda x: x.fun)[:10]:
#for r in res[:10]:
    print(r.fun,r.x)
    rmsd,cond,_,_,f = goodbase(r.x,M0,base,cen,opt=False)
    print(f,rmsd,cond)

In [ ]:
bestT1 = sorted(res,key=lambda x: x.fun)[0].x
_,_,bestbase,bestbase2,_ = goodbase(bestT1,M0,base,cen,opt=False)

plt.figure(figsize=(12,8))
for i,b in enumerate(bestbase.T):
    plt.plot(ti_seq,b + cen,label=str(i))

plt.legend()
plt.show()

In [ ]:
curvecoord(bestbase[:,5],e_vec[:,:10])

In [ ]:
base @ bestbase2[:,4] - bestbase[:,4]

In [ ]:
e_vec[:,0]

In [ ]:
def M0fitted(curve,cntr,obase,base):
    centered = curve - cntr
    ocoord,oproj = curveproj(centered,obase)
    
    M0s = np.linalg.solve(base,ocoord)
    
    return M0s, oproj

In [ ]:
plt.figure(figsize=(12,8))

col=['blue','orange','green','red','magenta','yellow']

for i,idx in enumerate([14,440,814,1388,2218]):
    M0s,proj = M0fitted(data[idx],cen,base,bestbase2)
    print(idx, M0s)
    plt.plot(ti_seq,data[idx],marker='.',color=col[i],ls='')
#    plt.plot(ti_seq,proj + cen,label=f"{idx} proj")
    plt.plot(ti_seq,(bestbase @  M0s) + cen, label=f"{idx} M0s", color=col[i])
    
plt.legend()
plt.show()

In [ ]:
# bestbase=e_vec[:,:6]

nbase = bestbase / np.linalg.norm(bestbase,axis=0)
plt.figure(figsize=(12,8))

for idx in [14,815,2218,17324]:
#for idx in [2218]:
    coord = curvecoord(data[idx]-cen,bestbase)
    print(idx,coord)
    plt.plot(ti_seq,data[idx],marker='.',color='k',ls='')
    plt.plot(ti_seq,(bestbase @ coord) + cen, label=str(idx))
    
#    ncoord = curvecoord(data[idx]-cen,nbase)
#    plt.plot(ti_seq,(nbase @ ncoord) + cen, label=str(idx))
    
plt.legend()
plt.show()

In [ ]:
curvecoord(data[idx]-cen,bestbase[:,0])

In [ ]:
nbase.T @ nbase

In [ ]:
np.linalg.cond(nbase)

In [ ]:
curvecoord(bestbase[:,0],base)

In [ ]:
data[2218]-cen

In [ ]:
np.dot(data[2218]-cen,bestbase[:,0])/(np.linalg.norm(bestbase[:,0])**2)

In [ ]:
for t in [
    [660,716,800,950,1180,1620],
    [600,716,800,950,1180,1620],
    [500,716,800,950,1180,1620],
    [660,716,800,950,1000,1620],
    [660,716,800,950,950,1620]
]:
    print (goodbase(t,base,cen))

In [ ]:
data[0]

In [ ]:
bnorm

In [ ]:
m = np.array([[1,0],[2,0]])
m

In [ ]:
np.linalg.cond(m)

In [ ]:
m=np.array([np.array([1,2]), np.array([3,4])])
m

Vraciame sa spat do voxelov, kde hladame suradnice nameraneho signalu v baze a ich chyby.

In [ ]:
import nibabel as nib
# 3813A, 3816A, 3432B or 3832A
dataset_name = "3813A"
file_name = "data/dataset/" + dataset_name + "_ep2d_se_til_shlomi_50-3000.nii"
nib_data = nib.load(file_name)
img_data = nib_data.get_fdata()
print(img_data.shape)

In [ ]:
def find_signal_coefs(decentr, base):
    coefs = []
    for i in range(len(base)):
        l = np.linalg.norm(base[i])
        coefs.append(sum(decentr * base[i]) / (l ** 2))

    return coefs

def reconstruct_from_base(centroid, base, coefs, n):
    rec = np.copy(centroid)
    for i in range(n):
        rec += coefs[i] * base[i]

    return rec

def mse(x, y):
    return np.square(np.subtract(x,y)).mean()

In [ ]:
base = np.copy(e_vec).T

test_data = data[79]
coefs = find_signal_coefs(test_data - cen, base)
rec = reconstruct_from_base(cen, base, coefs, 6)

In [ ]:
if True:
    all_coefs = np.load("data/dataset/coefs_in_base.npy")
    errors = np.load("data/dataset/errors.npy")
else:

    z_len, y_len, x_len, _ = img_data.shape

    for z in range(z_len):
        for y in range(y_len):
            for x in range(x_len):
                test_data = img_data[z][y][x]
                c = find_signal_coefs(test_data - cen, base)
                rec = reconstruct_from_base(cen, base, c, 6)
                errors[z][y][x] = mse(test_data, rec)

                all_coefs[z][y][x] = c
        print(f"{z} iteration finished")

    np.save("data/dataset/coefs_in_base", all_coefs)
    np.save("data/dataset/errors", errors)

In [ ]:
t_errors = np.transpose(errors.copy(), (2, 0, 1))
# norm e
# t_errors = (t_errors - np.min(t_errors)) / (np.max(t_errors) - np.min(t_errors))
print(t_errors.shape)
t_coefs = np.transpose(all_coefs.copy(), (3, 2, 0, 1))

In [ ]:
for slice in []: #t_errors:
    plt.imshow(slice.astype(int))
    plt.colorbar()
    plt.title("Errors")
    plt.show()

In [ ]:
print(t_coefs.shape)
for i in range(10, 30):
    plt.imshow(t_coefs[1][i].astype(int))
    plt.colorbar()
    plt.title("Errors")
    plt.show()




In [ ]:
m = np.array([[0.75,4.25,3],[0.5,0,0],[0,0.5,0]])
m = np.array([[0.,4.55,16.5],[0.2,0,0],[0,0.1,0]])

In [ ]:
l,v = np.linalg.eig(m)

In [ ]:
l

In [ ]:
v0 = v[:,0]

In [ ]:
v0 / v0[2]